In [1]:
import ccxt
import numpy as np
import pandas as pd
import datetime as dt
from dateutil import tz
import matplotlib.pyplot as plt

import json
import time
import os
import sys

In [2]:
src_path = '../src/'
sys.path.append(os.path.abspath(src_path))
from func_get import *
from func_signal import *
from func_backtest import *

In [3]:
with open('../../../_keys/ftx/ftx_read_keys.json') as keys_file:
    keys_dict = json.load(keys_file)
    
exchange = ccxt.ftx({'apiKey': keys_dict['apiKey'],
                     'secret': keys_dict['secret'],
                     'enableRateLimit': True})

### Config

In [4]:
symbol = 'ETH-PERP'
timeframe = '15m'

### Get data

In [5]:
ohlcv_df = get_ohlcv_df(exchange, symbol, timeframe, since=None, limit=1000)
ohlcv_df = ohlcv_df.loc[:len(ohlcv_df) - 2, :]

In [6]:
ohlcv_df

,time,open,high,low,close,volume
0,2022-02-18 14:15:00,2910.5,2917.6,2896.6,2899.1,2.742885e+07
1,2022-02-18 14:30:00,2898.9,2904.6,2896.3,2904.5,4.867893e+06
2,2022-02-18 14:45:00,2904.5,2906.5,2899.0,2905.8,4.151703e+06
3,2022-02-18 15:00:00,2905.8,2917.2,2904.3,2915.8,1.435746e+07
4,2022-02-18 15:15:00,2915.8,2923.3,2909.9,2913.7,1.227192e+07
...,...,...,...,...,...,...
994,2022-02-28 22:45:00,2804.1,2830.0,2800.5,2817.3,4.369428e+07
995,2022-02-28 23:00:00,2817.5,2833.2,2799.0,2805.2,7.960309e+07
996,2022-02-28 23:15:00,2805.2,2831.6,2801.3,2823.4,5.453287e+07
997,2022-02-28 23:30:00,2823.4,2829.9,2810.0,2822.1,3.916571e+07


### Signal

In [7]:
signal_dict = {
    'windows': 50,
    'std': 2,
    'revert': False
}

In [8]:
def add_bollinger(signal_dict, ohlcv_df):
    windows = signal_dict['windows']
    
    temp_df = ohlcv_df.copy()

    sma_list = temp_df['close'].rolling(window=windows).mean()
    std_list = temp_df['close'].rolling(window=windows).std(ddof=0)
    
    bollinger_upper = sma_list + (std_list * signal_dict['std'])
    bollinger_lower = sma_list - (std_list * signal_dict['std'])
    
    ohlcv_df['bollinger_upper'] = bollinger_upper
    ohlcv_df['bollinger_lower'] = bollinger_lower
    
    return ohlcv_df

In [9]:
ohlcv_df = add_bollinger(signal_dict, ohlcv_df)

In [10]:
ohlcv_df

,time,open,high,low,close,volume,bollinger_upper,bollinger_lower
0,2022-02-18 14:15:00,2910.5,2917.6,2896.6,2899.1,2.742885e+07,NaN,NaN
1,2022-02-18 14:30:00,2898.9,2904.6,2896.3,2904.5,4.867893e+06,NaN,NaN
2,2022-02-18 14:45:00,2904.5,2906.5,2899.0,2905.8,4.151703e+06,NaN,NaN
3,2022-02-18 15:00:00,2905.8,2917.2,2904.3,2915.8,1.435746e+07,NaN,NaN
4,2022-02-18 15:15:00,2915.8,2923.3,2909.9,2913.7,1.227192e+07,NaN,NaN
...,...,...,...,...,...,...,...,...
994,2022-02-28 22:45:00,2804.1,2830.0,2800.5,2817.3,4.369428e+07,2747.988464,2542.731536
995,2022-02-28 23:00:00,2817.5,2833.2,2799.0,2805.2,7.960309e+07,2760.730122,2537.489878
996,2022-02-28 23:15:00,2805.2,2831.6,2801.3,2823.4,5.453287e+07,2774.577115,2532.262885
997,2022-02-28 23:30:00,2823.4,2829.9,2810.0,2822.1,3.916571e+07,2787.082945,2527.505055
